In [2]:
# Description: test4
# Only get all games from Steam API
import aiohttp
import asyncio
import pandas as pd

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(games):
    games_data = [(game["appid"], game["name"]) for game in games["applist"]["apps"] if game['name']]
    df = pd.DataFrame(games_data, columns=["appid", "name"])
    df.to_csv("games.csv", index=False)

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(games)

if __name__ == "__main__":
    await main()


In [5]:
# (Working version) Combine all the data, both games and game details
import aiohttp
import asyncio
import pandas as pd
import time

async def fetch_game_details(session, appid):
    async with session.get(f"https://steamspy.com/api.php?request=appdetails&appid={appid}") as resp:
        return await resp.json()

async def fetch_games(session):
    async with session.get("https://api.steampowered.com/ISteamApps/GetAppList/v2") as resp:
        return await resp.json()

async def save_games_to_csv(session,games):
    games_data = []
    request_count = 0
    batch = 0
    batch_size = 0
    for game in games["applist"]["apps"][batch:150000]:
        game_details = await fetch_game_details(session, game["appid"])
        request_count += 1
        if request_count == 1000:
            time.sleep(60)
            request_count = 0
        if batch_size == 25000:
            df = pd.DataFrame(games_data, columns=["appid", "name", "price", "developer", "publisher", "average_forever", "median_forever", "ccu", "genres"])
            df.to_csv("games.csv", mode="a", header=False, index=False)
            games_data = []
            break
        if game_details:
            data = game_details
            games_data.append((game["appid"], game["name"], data.get("price", "0"), data.get("developer", ""), data.get("publisher", ""), data.get("average_forever", 0), data.get("median_forever", 0), data.get("ccu", 0), data.get("genre", [])))
            batch_size += 1
    '''''
    df = pd.DataFrame(games_data, columns=["appid", "name", "price", "developer", "publisher", "average_forever", "median_forever", "ccu", "genres"])
    df.to_csv("games.csv", index=False)
    return df
    '''''

async def main():
    async with aiohttp.ClientSession() as session:
        games = await fetch_games(session)
        await save_games_to_csv(session, games)

if __name__ == "__main__":
     await main()